In [1]:
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# Read in the data
df = pd.read_csv("Resources/runner_result_runner_result.csv")
df = df.drop(columns="Unnamed: 0")
df

,meetingName,meetingDate,raceNumber,runnerNumber,location,weatherCondition,trackCondition,raceName,raceStartTime,raceDistance,...,track (rider wins),region (rider wins),last30Days (rider wins),last12Months (rider wins),runner (rider wins),track (rider placings),region (rider placings),last30Days (rider placings),last12Months (rider placings),runner (rider placings)
0,STONY CREEK,01/02/2022 12:00:00 AM,1,3,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,0,7,12,135,0,0,6,33,229,0
1,STONY CREEK,01/02/2022 12:00:00 AM,1,11,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,0,0,0,0,0,0,0,0,0,0
2,STONY CREEK,01/02/2022 12:00:00 AM,1,2,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,2,16,6,38,0,3,21,11,104,0
3,STONY CREEK,01/02/2022 12:00:00 AM,1,9,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,2,12,8,93,0,1,28,11,176,1
4,STONY CREEK,01/02/2022 12:00:00 AM,1,8,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,0,8,3,56,0,7,18,1,110,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,YORK,19/05/2022 12:00:00 AM,8,5,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,0,2,7,57,0,2,7,11,147,0
13184,YORK,19/05/2022 12:00:00 AM,8,4,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,0,13,1,66,0,0,11,4,132,0
13185,YORK,19/05/2022 12:00:00 AM,8,7,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,1,7,3,28,0,6,12,3,56,0
13186,YORK,19/05/2022 12:00:00 AM,8,6,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,0,13,3,47,0,4,18,9,101,0


In [3]:
# Convert categorical features to numerical
weatherCondition = pd.get_dummies(df.weatherCondition, sparse=True)
trackCondition = pd.get_dummies(df.trackCondition, sparse=True)
meetingName = pd.get_dummies(df.meetingName, sparse=True)
runnerNumber = pd.get_dummies(df.runnerNumber, sparse=True)
raceClassConditions = pd.get_dummies(df.raceClassConditions, sparse=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13188 entries, 0 to 13187
Data columns (total 89 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   meetingName                      13188 non-null  object 
 1   meetingDate                      13188 non-null  object 
 2   raceNumber                       13188 non-null  int64  
 3   runnerNumber                     13188 non-null  int64  
 4   location                         0 non-null      float64
 5   weatherCondition                 13022 non-null  object 
 6   trackCondition                   13022 non-null  object 
 7   raceName                         13188 non-null  object 
 8   raceStartTime                    13188 non-null  object 
 9   raceDistance                     13188 non-null  int64  
 10  trackDirection                   0 non-null      float64
 11  raceClassConditions              13188 non-null  object 
 12  runnerName        

In [5]:
# Combine the features of interest into a dataframe
X = pd.concat([weatherCondition,
               trackCondition,
               meetingName,
               runnerNumber,
               raceClassConditions,
               df.raceDistance,
               df.finishingPosition
              ],axis=1)

print(f"Shape of feature vector before cleaning: {X.size}")
# Drop any nans (maybe check why they are there)
X = X.dropna()
print(f"Shape of feature vector after cleaning: {X.size}")

X.columns = X.columns.astype(str)


Shape of feature vector before cleaning: 3362940
Shape of feature vector after cleaning: 3320610


In [6]:
#pd.set_option('display.max_columns', None)
#print(X)

In [7]:
# Scale the data
scaled_data = StandardScaler().fit_transform(X)



/Users/darcy5d/miniforge3/envs/tensor/lib/python3.8/site-packages/sklearn/utils/validation.py:785: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/darcy5d/miniforge3/envs/tensor/lib/python3.8/site-packages/sklearn/utils/validation.py:785: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [8]:
k = list(range(1, 20))
inertia = []
for i in k:
    model = KMeans(n_clusters=i, random_state=0,n_init='auto')
    model.fit(scaled_data)
    inertia.append(model.inertia_)
    
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k,"inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [9]:
elbow_plot = df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)
elbow_plot

:Curve   [k]   (inertia)

In [10]:
import seaborn as sns

In [11]:
#sns.pairplot()

In [12]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=3)

# market_pca_data = pca.fit_transform(scaled_data)
# pca.explained_variance_ratio_

In [13]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=10, random_state=0, n_init='auto')

# Fit the K-Means model using the scaled data
model.fit(scaled_data)

# Predict the clusters to group the horses using the scaled data
horse_clusters = model.predict(scaled_data)

# View the resulting array of cluster values.
print(horse_clusters)

# Create a copy of the DataFrame
scaled_predictions = X.copy()

# Add a new column to the DataFrame with the predicted clusters
scaled_predictions["horse_clusters"] = horse_clusters

# Display sample data
scaled_predictions

[0 0 0 ... 6 6 6]


,FINE,OCAST,RAIN,SHWRY,AWT,FAST,GOOD,GOOD3,GOOD4,HVY10,...,RH-52,STEEPL,SWP-G3,WFA,WFA-G1,WFA-G2,WFA-G3,raceDistance,finishingPosition,horse_clusters
0,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,1100,4.0,0
1,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,1100,-2.0,0
2,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,1100,0.0,0
3,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,1100,-2.0,0
4,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,1100,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,1300,2.0,6
13184,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,1300,4.0,6
13185,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,1300,0.0,6
13186,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,1300,1.0,6


In [14]:
clusters_plot = scaled_predictions.hvplot.scatter(
    x="finishingPosition",
    y="raceDistance",
    by="horse_clusters",
)
clusters_plot

:NdOverlay   [horse_clusters]
   :Scatter   [finishingPosition]   (raceDistance)

In [15]:
# Convert finishingPosition to binary: 1 if the horse placed (1-4), 0 otherwise
X['finishingPosition'] = X['finishingPosition'].apply(lambda x: 1 if 1.0 <= x <= 3.0 else 0)


In [16]:
# things to do 


In [17]:
from sklearn.model_selection import train_test_split

# Features and target variable
y = X['finishingPosition']
X_features = X.drop('finishingPosition', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=1)

In [18]:
#X_features = X.drop('finishingPosition', axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2, random_state=1)

In [19]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize a Sequential model
model = Sequential()

# Add layers to the model
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(48, activation='relu'))  # Hidden layer
model.add(Dense(24, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [21]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
326/326 [==============================] - 0s 855us/step - loss: 0.6861 - accuracy: 0.7456 - val_loss: 0.6750 - val_accuracy: 0.6008
Epoch 2/50
326/326 [==============================] - 0s 636us/step - loss: 0.5649 - accuracy: 0.7769 - val_loss: 0.5242 - val_accuracy: 0.7846
Epoch 3/50
326/326 [==============================] - 0s 621us/step - loss: 0.5530 - accuracy: 0.7741 - val_loss: 0.5307 - val_accuracy: 0.7846
Epoch 4/50
326/326 [==============================] - 0s 618us/step - loss: 0.5356 - accuracy: 0.7804 - val_loss: 0.6653 - val_accuracy: 0.5785
Epoch 5/50
326/326 [==============================] - 0s 621us/step - loss: 0.5633 - accuracy: 0.7714 - val_loss: 0.8688 - val_accuracy: 0.7846
Epoch 6/50
326/326 [==============================] - 0s 626us/step - loss: 0.5572 - accuracy: 0.7774 - val_loss: 0.5284 - val_accuracy: 0.7846
Epoch 7/50
326/326 [==============================] - 0s 618us/step - loss: 0.5349 - accuracy: 0.7854 - val_loss: 0.6049 - val_accuracy:

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


82/82 [==============================] - 0s 326us/step - loss: 0.5213 - accuracy: 0.7846
Test Loss: 0.5213
Test Accuracy: 0.7846


In [23]:
print(confusion_matrix(y_test, y_pred))

NameError: name 'confusion_matrix' is not defined

In [24]:
model.save("my_model.h5")

/Users/darcy5d/miniforge3/envs/tensor/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

In [ ]:
rf_model = rf_model.fit(X_train, y_train)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
importances = rf_model.feature_importances_
feature_importance = pd.DataFrame({
    'Feature': X_train.columns, 
    'Importance': importances
})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)


In [ ]:
import matplotlib.pyplot as plt

# Sort feature importances in descending order and plot
sorted_idx = importances.argsort()[::-1]
plt.figure(figsize=(10, 12))
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")
plt.show()


In [ ]:
print(feature_importance[0:30])

In [ ]:
feature_importance['Feature'][:20].to_list()

In [ ]:
X = X[feature_importance['Feature'][:20].to_list()]